In [20]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [4]:
#Định dạng lại ảnh đầu vào
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [5]:
filenames = os.listdir("training_data_set")

In [6]:
categories=[]

In [7]:
#Định nghĩa danh mục và tên
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})
df

,filename,category
0,cat.2001.jpg,0
1,cat.2002.jpg,0
2,cat.2003.jpg,0
3,cat.2004.jpg,0
4,cat.2005.jpg,0
...,...,...
3995,dog.3996.jpg,1
3996,dog.3997.jpg,1
3997,dog.3998.jpg,1
3998,dog.3999.jpg,1


In [8]:
#Khởi tạo mạng nơ ron và các lớp Layer
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])


In [9]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=20

train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "training_data_set",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    "training_data_set",
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)



Found 3200 validated image filenames belonging to 2 classes.
Found 800 validated image filenames belonging to 2 classes.


In [10]:
epochs=10
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)



C:\Users\caoqu\AppData\Local\Temp\ipykernel_3832\1328075405.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
160/160 [==============================] - 310s 2s/step - loss: 1.1977 - accuracy: 0.5581 - val_loss: 2.9028 - val_accuracy: 0.5275 - lr: 0.0010
Epoch 2/10
160/160 [==============================] - 265s 2s/step - loss: 0.8313 - accuracy: 0.5831 - val_loss: 1.7454 - val_accuracy: 0.5325 - lr: 0.0010
Epoch 3/10
160/160 [==============================] - 291s 2s/step - loss: 0.7196 - accuracy: 0.6203 - val_loss: 0.8323 - val_accuracy: 0.6100 - lr: 0.0010
Epoch 4/10
160/160 [==============================] - 273s 2s/step - loss: 0.6375 - accuracy: 0.6578 - val_loss: 0.7683 - val_accuracy: 0.6500 - lr: 0.0010
Epoch 5/10
160/160 [==============================] - 286s 2s/step - loss: 0.6113 - accuracy: 0.6741 - val_loss: 0.6651 - val_accuracy: 0.6388 - lr: 0.0010
Epoch 6/10
160/160 [==============================] - 288s 2s/step - loss: 0.5968 - accuracy: 0.6844 - val_loss: 0.5237 - val_accuracy: 0.7312 - lr: 0.0010
Epoch 7/10
160/160 [==============================] - 283s 2s/st

In [11]:
model.save("Model1.keras")

In [30]:
# test model
test_filenames = os.listdir("./test_data_set")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]
# Định nghĩa generator
test_generator = test_data_generator.flow_from_dataframe(
  dataframe=test_df, 
  x_col="filename",
  batch_size=32
)

# Dự đoán
predict = model.predict_generator(
  test_generator,
  steps=np.ceil(len(test_df)/batch_size)
)

test_df['category'] = np.argmax(predict, axis=-1)
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("./dogs-vs-cats/test1/"+filename, target_size=Image_Size)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()


results={
    0:'cat',
    1:'dog'
}
from PIL import Image
import numpy as np
im=Image.open("__image_path_TO_custom_image")
im=im.resize(Image_Size)
im=np.expand_dims(im,axis=0)
im=np.array(im)
im=im/255
pred=model.predict_classes([im])[0]
print(pred,results[pred])

KeyError: 'class'